In [ ]:
from pathlib import Path

import cantera
import polars
from cantera.ck2yaml import Parser

from cantera_helper.reactors import jsr

# >> Configure these parameters as needed: <<
# Required files:
concentrations_file = "concentrations.csv"  # Concentrations spreadsheet
species_file = "species.csv"
chemkin_file = "chem.inp"  # Chemkin mechanism
thermo_file = "thermo.dat"
# Generated files:
cantera_file = Path(chemkin_file).with_suffix(".yaml")  # Cantera mechanism
results_file = "results.csv"  # Results will be written to this file
# Conditions / other parameters:
T = 825  # Temperature (K)
P = 1.1  # Pressure (atm)
residence_time = 2  # Residence time (s)
volume = 1  # Volume (cm^3)
run_every = 15  # Run every nth concentration for testing
# run_every = 1  # (Set this to 1 for the actual simulation)

In [2]:
# Read in concentrations
print(f"Reading in concentrations from {concentrations_file}")
concs_df = polars.read_csv(concentrations_file)
concs_df = concs_df.gather_every(run_every)
conc_dcts = concs_df.select("fuel", "O2", "N2").to_dicts()

Reading in concentrations from concentrations.csv


In [3]:
# Read in species
print(f"Reading in species from {species_file}")
species_df = polars.read_csv(species_file)
species_dct = {k: v for k, v in species_df.select(["species", "name"]).iter_rows()}

Reading in species from species.csv


In [4]:
conc_dcts = [{species_dct[k]: v for k, v in c.items()} for c in conc_dcts]
species_dct = {k: v for k, v in species_dct.items() if k not in ["fuel", "O2", "N2"]}

In [5]:
# Determine N2, O2, fuel names
print("Simulations will be run at the following conditions...")
for conc_dct in conc_dcts:
    for name, conc_dct in conc_dct.items():
        print(f"  {name}: {conc_dct:10.6f}", end="")
    print()

Simulations will be run at the following conditions...
  C5H8(522):   0.005000  O2(6):   0.005833  N2:   0.989167
  C5H8(522):   0.005000  O2(6):   0.087500  N2:   0.907500


In [6]:
# Print other species names
print("Data will be collected for the following species...")
for species, name in species_dct.items():
    print(f"  {species}: {name}")

Data will be collected for the following species...
  CO: CO(12)
  CO2: CO2(13)
  CH4: CH4(33)
  H2: H2(2)
  oxygen: O2(6)
  ethene: C2H4(52)
  formaldehyde: CH2O(19)
  propene: C3H6(131)
  water: H2O(5)
  propadiene (allene): C3H4(114)
  acetaldehyde: CH3CHO(41)
  1,3-butadiene: C4H6(227)
  1-butene: C4H8(253)
  acrolein: C3H4O(165)
  propanal: C3H6O(126)
  furan: S(1600)
  cyclopentadiene: C5H6(478)
  cyclopentene: C5H8(522)
  cyclopentane: CPT(563)
  crotonaldehyde (trans): C4H6O(357)
  crotonaldehyde (cis): C4H6O(357)
  benzene: C6H6(970)
  1,2-epoxycyclopentane: C5H8O(825)
  toluene: C7H8(1006)
  cyclopentanone: CPN(801)
  cyclopent-2-enone: CPND2(626)
  3-methylfuran: S(1657)


In [7]:
# Read in ChemKin mechanism and convert to Cantera
print("\nConverting ChemKin mechanism to Cantera YAML...")
Parser.convert_mech(chemkin_file, thermo_file=thermo_file, out_name=cantera_file)


Converting ChemKin mechanism to Cantera YAML...


Wrote YAML mechanism file to 'chem.yaml'.
Mechanism contains 2534 species and 10002 reactions.


(<cantera.ck2yaml.Parser at 0x76994953ecf0>, [])

In [8]:
# Load mechanism and set initial conditions
print("\nDefining model and conditions...")
model = cantera.Solution(cantera_file)


Defining model and conditions...


In [9]:
# Run simulations for each point and store the results in an array
print("\nRunning simulations...")


def simulate(conc_dct: dict[str, float]) -> cantera.ReactorNet:  # pyright: ignore[reportAttributeAccessIssue]
    """Run simulation."""
    print(f"Starting simulation for {conc_dct}")
    return jsr(
        model=model,
        T=T,
        P=P,
        residence_time=residence_time,
        volume=volume,
        concentrations=conc_dct,
    )


solutions = cantera.SolutionArray(model)
for conc_dct in conc_dcts:
    reactor = simulate(conc_dct)
    solutions.append(reactor.thermo.state)


Running simulations...
Starting simulation for {'C5H8(522)': 0.005, 'O2(6)': 0.005833333, 'N2': 0.989166667}
Starting simulation for {'C5H8(522)': 0.005, 'O2(6)': 0.0875, 'N2': 0.9075}


In [10]:
# Extract results
print("\nExtracting results...")
results_df = concs_df.with_columns(
    polars.Series(species, solutions(name).X.flatten() * 10**6)
    for species, name in species_dct.items()
)
print(results_df)
results_df.write_csv(results_file)


Extracting results...
shape: (2, 31)
┌─────┬──────────┬──────────┬───────┬───┬──────────┬───────────────┬───────────────┬───────────────┐
│ phi ┆ O2       ┆ N2       ┆ fuel  ┆ … ┆ toluene  ┆ cyclopentanon ┆ cyclopent-2-e ┆ 3-methylfuran │
│ --- ┆ ---      ┆ ---      ┆ ---   ┆   ┆ ---      ┆ e             ┆ none          ┆ ---           │
│ f64 ┆ f64      ┆ f64      ┆ f64   ┆   ┆ f64      ┆ ---           ┆ ---           ┆ f64           │
│     ┆          ┆          ┆       ┆   ┆          ┆ f64           ┆ f64           ┆               │
╞═════╪══════════╪══════════╪═══════╪═══╪══════════╪═══════════════╪═══════════════╪═══════════════╡
│ 6.0 ┆ 0.005833 ┆ 0.989167 ┆ 0.005 ┆ … ┆ 0.000032 ┆ 0.00172       ┆ 0.114572      ┆ 8.6833e-7     │
│ 0.4 ┆ 0.0875   ┆ 0.9075   ┆ 0.005 ┆ … ┆ 0.000934 ┆ 0.388524      ┆ 27.6619       ┆ 0.000564      │
└─────┴──────────┴──────────┴───────┴───┴──────────┴───────────────┴───────────────┴───────────────┘
